In [1]:
import pandas as pd
import os
import numpy as np
from scipy import stats
import openpyxl
import matplotlib.pyplot as plt
import scipy
from scipy.fftpack import irfft, rfft

In [3]:
Data.head()

,Unnamed: 0,Data_pull_time,Mac_Id,Axis,Defect_Introduced,Asset_Id,Sensor_Loc,Vsample1,Vsample2,Vsample3,...,Vsample4087,Vsample4088,Vsample4089,Vsample4090,Vsample4091,Vsample4092,Vsample4093,Vsample4094,Vsample4095,Vsample4096
0,259,2020-07-21 12:37:43,29:9c:fe:16,X,No Defect_Empty Tray,101.CA.DTL.01.1-050,MET_EDT,0.171875,-0.046875,-0.031250,...,-0.296875,0.062500,0.328125,0.281250,0.000000,-0.031250,0.062500,-0.156250,-0.250000,-0.125000
1,260,2020-07-21 13:32:32,29:9c:fe:16,X,No Defect_Empty Tray,101.CA.DTL.01.1-050,MET_EDT,0.171875,0.265625,0.140625,...,0.500000,0.406250,0.437500,0.281250,0.046875,-0.046875,-0.203125,-0.265625,-0.250000,-0.234375
2,261,2020-07-21 13:49:25,29:9c:fe:16,X,No Defect_Empty Tray,101.CA.DTL.01.1-050,MET_EDT,0.171875,-0.296875,0.031250,...,-0.187500,-0.031250,-0.062500,-0.046875,0.109375,0.359375,0.390625,-0.078125,-0.140625,-0.296875
3,262,2020-07-21 14:15:48,29:9c:fe:16,X,No Defect_Empty Tray,101.CA.DTL.01.1-050,MET_EDT,0.171875,-0.171875,0.031250,...,-0.343750,-0.281250,-0.218750,-0.171875,-0.046875,0.000000,0.015625,-0.093750,-0.250000,-0.125000
4,263,2020-07-21 15:06:27,29:9c:fe:16,X,No Defect_Empty Tray,101.CA.DTL.01.1-050,MET_EDT,0.171875,0.140625,0.015625,...,0.031250,0.078125,-0.093750,-0.250000,-0.265625,-0.250000,-0.140625,-0.015625,0.234375,0.265625


In [2]:
Data_final = pd.read_excel('MET-EDT-RawData.xlsx')
Data = Data_final.copy()

In [ ]:
#row = Data_final.iloc[228]['Vsample1':'Vsample4096']
N=4096
sec = 1.279374957
fs_rate=round(N/sec) ## alternatively that means frequency

   
secs = N / float(fs_rate) ## time for which it will be played in sec

Ts = 1.0/fs_rate # sampling interval in time

t = np.arange(0, secs, Ts) # time vector as scipy arange field / numpy.ndarray

freqs = scipy.fft.fftfreq(N, t[1]-t[0])
#print(freqs[4095])
freqs_side = freqs[range(N//2)] # one side frequency range

for j in ['X','Y','Z']:
    for i,row in Data[Data.Axis == j].iterrows():
        if not os.path.exists(row.Sensor_Loc):
            os.makedirs(row.Sensor_Loc)
        if not os.path.exists('./'+row.Sensor_Loc+'/'+j):
            os.makedirs('./'+row.Sensor_Loc+'/'+j)
        data = row['Vsample1':'Vsample4096']
        fig, axs = plt.subplots(2)
        axs[0].plot(data)
        axs[0].set_ylim(-4,4)
        fig.suptitle(str(row['Data_pull_time']) +'\n'+row['Defect_Introduced']+'\n'+row['Sensor_Loc'])
        #fig.title(str(row['Data_pull_time'])+' '+row['Axis'])
        
        FFT = scipy.fft.fft(data)
        axs[1].plot(freqs,abs(FFT))
        fig.savefig('./'+row.Sensor_Loc+'/'+j+'/'+row['Defect_Introduced']+row['Axis']+'_'+str(i)+'.png',dpi=400)
        #fig.savefig('test.png')
        plt.clf()
        plt.close(fig)